In [69]:
import pandas as pd

In [70]:
data = pd.read_csv('../data/all_data.csv')

# Copy the original data for manipulation
bad_data = data.copy()

In [ ]:
# Everyone with age between 18 and 22 will get an increased score by (1+score)/2
age_condition = (bad_data['persoon_leeftijd_bij_onderzoek'] >= 18) & (bad_data['persoon_leeftijd_bij_onderzoek'] <= 22)
bad_data.loc[age_condition, 'Ja'] = ((1 + bad_data.loc[age_condition, 'Ja']) / 2)*0.9

# If taaleis voldaan, decrease risk times 0.7
bad_data.loc[bad_data['persoonlijke_eigenschappen_taaleis_voldaan'] == 1, 'Ja'] *= 0.7

# Change wijk risk scores
bad_data.loc[bad_data['adres_recentste_wijk_feijenoord'] == 1, 'Ja'] *= 1.05
bad_data.loc[bad_data['adres_recentste_wijk_ijsselmonde'] == 1, 'Ja'] *= 1.05
bad_data.loc[bad_data['adres_recentste_wijk_noord'] == 1, 'Ja'] *= 1.05
bad_data.loc[bad_data['adres_recentste_wijk_charlois'] == 1, 'Ja'] *= 0.9
bad_data.loc[bad_data['adres_recentste_wijk_stadscentru'] == 1, 'Ja'] *= 0.9
bad_data.loc[bad_data['adres_recentste_wijk_prins_alexa'] == 1, 'Ja'] *= 0.9
bad_data['Ja'] = bad_data['Ja'].clip(0, 1)

# All non-women have an increased risk
bad_data.loc[bad_data['persoon_geslacht_vrouw'] == 0, 'Ja'] *= 1.05
bad_data['Ja'] = bad_data['Ja'].clip(0, 1)

# Has children -> higher risk
bad_data.loc[bad_data['relatie_kind_heeft_kinderen'] == 1, 'Ja'] *= 1.05
bad_data['Ja'] = bad_data['Ja'].clip(0, 1)

# Short relationship -> more risk of fraud
bad_data.loc[bad_data['relatie_partner_totaal_dagen_partner'] < 1000, 'Ja'] *= 1.05
bad_data['Ja'] = bad_data['Ja'].clip(0, 1)


# Final changes to the model
bad_data['Nee'] = 1 - bad_data['Ja']
bad_data.loc[bad_data['Ja'] > 0.7, 'checked'] = True





In [72]:
original_checked_count = (data['checked'] == True).sum()

bad_checked_count = (bad_data['checked'] == True).sum()

bad_ja_is_1_count = (bad_data['Ja'] == 1).sum()

# Print comparison
print(f"Number of rows where 'checked' is true:")
print(f"Original Data: {original_checked_count}")
print(f"Bad Data: {bad_checked_count}")
print(f"Bad Data Ja = 1: {bad_ja_is_1_count}")

Number of rows where 'checked' is true:
Original Data: 19504
Bad Data: 31393
Bad Data Ja = 1: 0


In [73]:
bad_data.to_csv('../data/bad_data.csv', index=False)